In [13]:
import numpy as np
import pandas as pd

#X = onehotencoder.fit_transform(imagesArray.reshape(-1,1)).toarray()
#y = dataSet["objeto"]
from keras.utils.np_utils import to_categorical
ames = pd.read_csv('ames.csv', nrows= 1e6)


In [14]:
np.random.seed(42)


variables_independientes = ames.drop("precio_venta", axis=1).columns
datos_numericos = ames[variables_independientes].select_dtypes([int, float, "float64","int64"])
col_no_numericas = ames[variables_independientes].select_dtypes([object]).columns
col_numericas = datos_numericos.columns

dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
     "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = ames[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = ames[col_categoricas]



In [15]:
from sklearn.preprocessing import  Normalizer
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
datos_imputer = imp_mean.fit_transform(datos_numericos);

transformer = Normalizer().fit(datos_imputer)  # fit does nothing.
dator_normalizados = transformer.transform(datos_imputer)
datos_numericos_imputados_normalizados = pd.DataFrame(
    dator_normalizados,
    columns=datos_numericos.columns
)

In [16]:
datos_categoricos_dummy = pd.get_dummies(datos_categoricos, drop_first=True)

In [17]:
for columna_ordinal, valores in dict_var_ordinales.items():
    datos_ordinales[columna_ordinal] = (
    datos_ordinales[columna_ordinal]
    .astype("category")
    .cat.set_categories(valores)
    .cat.codes
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:

ames_procesado = pd.concat([
    datos_numericos_imputados_normalizados,
    datos_categoricos_dummy,
    datos_ordinales
], axis=1)
variable_objetivo = ["precio_venta"]
variables = ["area_parcela","area_piscina","n_chimeneas","n_cocinas","n_dormitorios_sobre_suelo","area_habitable_sobre_suelo", "calidad_general"]
ames_X = ames_procesado[variables]
ames_y = ames[variable_objetivo]





In [24]:



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test ,y_train,  y_test = train_test_split(ames_X, ames_y)
modelo = LinearRegression()
modelo.fit(X=X_train,
           y=y_train)

y_pred = modelo.predict(X_test)

#print(X_train, X_test)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print(mse)

[[ 98201.52415951]
 [205880.24491542]
 [133311.96894517]
 [145910.8628455 ]
 [121920.73575088]
 [157740.17113279]
 [307150.83066462]
 [208056.26251425]
 [133819.79801386]
 [156764.93110382]
 [181123.16380103]
 [227692.71138424]
 [218043.95230957]
 [176816.69582138]
 [206859.22722061]
 [130612.50051818]
 [229570.90784788]
 [214790.635229  ]
 [ 83156.62072479]
 [123538.62632329]
 [179102.12435663]
 [179840.45549009]
 [155531.60285709]
 [155304.90908533]
 [201946.62408179]
 [119705.84408483]
 [427259.91675218]
 [ 44695.31129233]
 [204285.36486939]
 [219815.0244692 ]
 [153488.91845806]
 [121551.90552858]
 [174103.73732675]
 [101379.26655386]
 [145177.63819105]
 [171946.18860617]
 [179346.2931684 ]
 [131979.15342348]
 [129969.06165404]
 [159558.02795503]
 [182232.09844788]
 [137958.90515494]
 [131023.42847059]
 [ 85851.29324779]
 [274717.25316011]
 [167317.40395865]
 [145177.63701926]
 [129401.90304986]
 [173959.15194033]
 [127420.7782378 ]
 [132111.5319861 ]
 [264875.00088304]
 [162106.464

In [25]:
from sklearn.externals import joblib

joblib.dump(modelo, 'modelo_ames.pkl') 
import json
with open('columnas_names.json', 'w') as fname:
    x_columnas = ames_X.columns.tolist()
    json.dump(x_columnas, fname)
    
x_dtypes = ames_X.dtypes
x_dtypes = {col: ames_X[col].dtype for col in ames_X.columns}
joblib.dump(x_dtypes, 'dtypes_ames.pkl')

['dtypes_ames.pkl']

,area_parcela,area_piscina,n_chimeneas,n_cocinas,n_dormitorios_sobre_suelo,area_habitable_sobre_suelo,calidad_general
0,31770,0,2,1,3,1656,6
1,11622,0,0,1,2,896,5
2,14267,0,0,1,3,1329,6
3,11160,0,2,1,3,2110,7
4,13830,0,1,1,3,1629,5
...,...,...,...,...,...,...,...
2925,7937,0,0,1,3,1003,6
2926,8885,0,0,1,2,902,5
2927,10441,0,0,1,3,970,5
2928,10010,0,1,1,2,1389,5
